In [1]:
pip install torch torchvision scikit-learn opencv-python tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import os

file_path = "/content/drive/MyDrive/New folder.zip"
if os.path.exists(file_path):
    print("✅ ZIP file mil gayi:", file_path)
else:
    print("❌ ZIP file nahi mili:", file_path)


✅ ZIP file mil gayi: /content/drive/MyDrive/New folder.zip


In [9]:
!unzip "/content/drive/MyDrive/New folder.zip" -d "/content/sample_data"


Archive:  /content/drive/MyDrive/New folder.zip
  inflating: /content/sample_data/New folder/IMG_0219.JPG  
  inflating: /content/sample_data/New folder/IMG_0220.JPG  
  inflating: /content/sample_data/New folder/IMG_0221.JPG  
  inflating: /content/sample_data/New folder/IMG_0222.JPG  
  inflating: /content/sample_data/New folder/IMG_0223.JPG  
  inflating: /content/sample_data/New folder/IMG_0301.JPG  
  inflating: /content/sample_data/New folder/IMG_0302.JPG  
  inflating: /content/sample_data/New folder/IMG_0303.JPG  
  inflating: /content/sample_data/New folder/IMG_0304.JPG  
  inflating: /content/sample_data/New folder/IMG_0305.JPG  
  inflating: /content/sample_data/New folder/IMG_0310.JPG  
  inflating: /content/sample_data/New folder/IMG_0311.JPG  
  inflating: /content/sample_data/New folder/IMG_0312.JPG  
  inflating: /content/sample_data/New folder/IMG_0313.JPG  
  inflating: /content/sample_data/New folder/IMG_0314.JPG  
  inflating: /content/sample_data/New folder/IMG_031

In [10]:
import torch
import torchvision.models as models

model = models.resnet50(pretrained=True)
model.eval()
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 97.0MB/s]


In [11]:
import torchvision.transforms as transforms
import cv2

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return transform(img).unsqueeze(0)


In [12]:
import numpy as np

def get_feature_vector(img_path):
    img_tensor = preprocess_image(img_path)
    with torch.no_grad():
        features = feature_extractor(img_tensor).squeeze().numpy()
    return features / np.linalg.norm(features)


In [13]:
from sklearn.metrics.pairwise import cosine_similarity

def pair_images(image_paths, features):
    pairs = []
    used = set()
    for i, img1 in enumerate(image_paths):
        if img1 in used:
            continue
        best_match = None
        best_score = 0
        for j, img2 in enumerate(image_paths):
            if img1 == img2 or img2 in used:
                continue
            sim = cosine_similarity([features[img1]], [features[img2]])[0][0]
            if sim > best_score:
                best_score = sim
                best_match = img2
        if best_match:
            pairs.append((img1, best_match))
            used.add(img1)
            used.add(best_match)
    return pairs


In [14]:
def classify_room(image_path):
    name = image_path.lower()
    if "kitchen" in name:
        return "Kitchen"
    elif "living" in name:
        return "Living Room"
    elif "closet" in name:
        return "Closet"
    else:
        return "Room"


In [15]:
import os
import shutil

def save_pairs(pairs, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for i, (before, after) in enumerate(pairs):
        label = classify_room(before + after)
        room_folder = os.path.join(output_folder, label)
        os.makedirs(room_folder, exist_ok=True)
        shutil.copy(before, os.path.join(room_folder, f"{label.lower()}_{i}_before.jpg"))
        shutil.copy(after, os.path.join(room_folder, f"{label.lower()}_{i}_after.jpg"))


In [21]:
from tqdm import tqdm

def organize_images(input_folder, output_folder):
    image_paths = []
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_paths.append(os.path.join(root, file))

    print(f"✅ Found {len(image_paths)} images.")
    print("📸 First few images:", image_paths[:5])

    features = {img: get_feature_vector(img) for img in tqdm(image_paths, desc="Extracting Features")}
    pairs = pair_images(image_paths, features)
    save_pairs(pairs, output_folder)




In [22]:
organize_images("/content/sample_data/New folder", "/content/Organized_Project")


✅ Found 47 images.
📸 First few images: ['/content/sample_data/New folder/IMG_7456.JPG', '/content/sample_data/New folder/IMG_0311.JPG', '/content/sample_data/New folder/IMG_7461.JPG', '/content/sample_data/New folder/IMG_0221.JPG', '/content/sample_data/New folder/IMG_0326.JPG']


Extracting Features: 100%|██████████| 47/47 [00:12<00:00,  3.79it/s]


In [23]:
!pip install timm --quiet


In [24]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import timm
import json
import urllib.request

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = timm.create_model('convnext_tiny.fb_in22k', pretrained=True)
model.eval().to(device)

# Load class index labels
url = 'https://storage.googleapis.com/download.tensorflow.org/data/imagenet_class_index.json'
with urllib.request.urlopen(url) as f:
    idx_to_labels = json.load(f)

ROOM_LABELS = {
    "kitchen": ["kitchen", "oven", "stove", "microwave"],
    "living room": ["living", "tv", "sofa", "couch"],
    "closet": ["closet", "wardrobe", "clothes"],
    "bathroom": ["bathroom", "toilet", "sink", "shower"],
    "bedroom": ["bed", "pillow"],
}

def predict_room_type(image_path):
    try:
        image = Image.open(image_path).convert('RGB')
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])
        img_tensor = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            outputs = model(img_tensor)
        _, predicted = outputs.max(1)
        label_text = idx_to_labels[str(predicted.item())][1].lower()
        for room, keywords in ROOM_LABELS.items():
            if any(keyword in label_text for keyword in keywords):
                return room.capitalize()
        return "Other"
    except:
        return "Other"


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/178M [00:00<?, ?B/s]

In [25]:
def save_pairs(pairs, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for i, (before, after) in enumerate(pairs):
        room_type = predict_room_type(before)
        room_folder = os.path.join(output_folder, room_type)
        os.makedirs(room_folder, exist_ok=True)
        shutil.copy(before, os.path.join(room_folder, f"{room_type.lower()}_{i}_before.jpg"))
        shutil.copy(after, os.path.join(room_folder, f"{room_type.lower()}_{i}_after.jpg"))


In [26]:
organize_images("/content/sample_data/New folder", "/content/Organized_Project")


✅ Found 47 images.
📸 First few images: ['/content/sample_data/New folder/IMG_7456.JPG', '/content/sample_data/New folder/IMG_0311.JPG', '/content/sample_data/New folder/IMG_7461.JPG', '/content/sample_data/New folder/IMG_0221.JPG', '/content/sample_data/New folder/IMG_0326.JPG']


Extracting Features: 100%|██████████| 47/47 [00:11<00:00,  3.99it/s]


In [27]:
import torch
import torchvision.transforms as transforms
from PIL import Image
from torchvision import models

# Load a pre-trained model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)
model = model.to(device)
model.eval()

# Dummy room labels – You can improve this by fine-tuning
room_keywords = {
    "kitchen": ["kitchen", "microwave", "oven", "refrigerator"],
    "bedroom": ["bed", "pillow", "blanket"],
    "bathroom": ["toilet", "sink", "bath"],
    "living_room": ["sofa", "television", "lamp"],
    "other": []
}

def classify_room_type(image_path):
    input_image = Image.open(image_path).convert("RGB")
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ])
    input_tensor = preprocess(input_image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(input_tensor)
        _, predicted = torch.max(outputs, 1)

    # Get class label from ImageNet
    class_idx = predicted.item()
    label = models.detection.IMAGE_NET_CLASSES[class_idx].lower()

    for room, keywords in room_keywords.items():
        if any(keyword in label for keyword in keywords):
            return room
    return "other"


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 67.2MB/s]


In [31]:
def guess_room_type(image_path):
    filename = os.path.basename(image_path).lower()

    if "kitchen" in filename or "031" in filename:
        return "Kitchen"
    elif "living" in filename or "032" in filename:
        return "Living Room"
    elif "bed" in filename or "745" in filename:
        return "Bedroom"
    else:
        return "Unknown"

def save_pairs_by_room_type(pairs, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for before_path, after_path in pairs:
        room_type = guess_room_type(before_path)
        room_folder = os.path.join(output_folder, room_type)
        os.makedirs(room_folder, exist_ok=True)

        # Copy files with proper naming
        shutil.copy2(before_path, os.path.join(room_folder, f"before_{os.path.basename(before_path)}"))
        shutil.copy2(after_path, os.path.join(room_folder, f"after_{os.path.basename(after_path)}"))

    print("✅ Images saved by room type.")

# ✅ Call this after pairing
save_pairs_by_room_type(pairs, "/content/Organized_Project")


✅ Images saved by room type.


In [34]:
import os
import json

output_folder = "/content/Organized_Project"

# Dictionary to hold final result
organized_data = {}

# Loop through subfolders like Kitchen, Living Room, etc.
for room_type in os.listdir(output_folder):
    room_path = os.path.join(output_folder, room_type)
    if os.path.isdir(room_path):
        files = sorted(os.listdir(room_path))
        before_img = [f for f in files if 'before' in f.lower()]
        after_img = [f for f in files if 'after' in f.lower()]

        # Pair up images and store
        pairs = list(zip(before_img, after_img))
        organized_data[room_type] = []

        for b, a in pairs:
            organized_data[room_type].append({
                "before": b,
                "after": a,
                "title": room_type
            })

# Save the JSON file
json_path = os.path.join(output_folder, "image_pairs_by_room.json")
with open(json_path, "w") as f:
    json.dump(organized_data, f, indent=4)

print(f"✅ JSON saved to: {json_path}")


✅ JSON saved to: /content/Organized_Project/image_pairs_by_room.json
